In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
import pandas as pd

In [5]:
#chromedriver_path = "C:\Program Files (x86)\chromedriver.exe"

In [34]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
# specify the url strings for the company's job posting website
url1 = 'https://careers.google.com/jobs/results/?page='
url2 = ''

In [33]:
def google_get_jobs(url1, url2):
    """
    retrieve job titles and job links from each page
    """
    job_title=[]
    job_link=[]
    page_num = 0

    driver=webdriver.Chrome(options = chrome_options)
    URL = url1 + str(page_num) + url2
    driver.get(URL)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # will exit while loop when soup.find_all returns None
    while soup.find_all("h2", {"class": "gc-card__title gc-heading gc-heading--beta"}) :
        job_title.extend([td.text for td in soup.findAll("h2", {"class": "gc-card__title gc-heading gc-heading--beta"})])
        job_link.extend(['https://careers.google.com' + td['href'] for td in soup.findAll("a", {"class": "gc-card"})])
        driver.quit()

        page_num += 1
        driver=webdriver.Chrome(options = chrome_options)
        URL = url1 + str(page_num) + url2
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit() 
    return job_title,job_link

In [13]:
job_title, job_link = google_get_jobs(url1, url2)

In [16]:
def remove_space(s):
  s_new = s.replace('\n', '')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [17]:
job_title = [remove_space(job) for job in job_title]

In [19]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

In [20]:
df_title_link.head()

,JOB_TITLE,JOB_LINK
0,"Software Engineer, Infrastructure, Looker Pro...",https://careers.google.com/jobs/results/920102...
1,Regional Legal Counsel,https://careers.google.com/jobs/results/825474...
2,"Scaled Customer Engineer, Google Cloud (Polis...",https://careers.google.com/jobs/results/107563...
3,"Technical Program Manager Intern, Summer 2023",https://careers.google.com/jobs/results/755884...
4,"Senior Android Software Engineer, Fitbit Devi...",https://careers.google.com/jobs/results/136197...


In [ ]:
# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

In [21]:
# save the titles and links to csv
df_title_link.to_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\google_jobs_titles_links_all.csv')

## Extract job descriptions and qualifications for each job

In [35]:
# load in the job titles and links from the csv file
df_title_link = pd.read_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\google_jobs_titles_links_all.csv', index_col=0)
df_title_link.head()

,JOB_TITLE,JOB_LINK
0,"Software Engineer, Infrastructure, Looker Pro...",https://careers.google.com/jobs/results/920102...
1,Regional Legal Counsel,https://careers.google.com/jobs/results/825474...
2,"Scaled Customer Engineer, Google Cloud (Polis...",https://careers.google.com/jobs/results/107563...
3,"Technical Program Manager Intern, Summer 2023",https://careers.google.com/jobs/results/755884...
4,"Senior Android Software Engineer, Fitbit Devi...",https://careers.google.com/jobs/results/136197...


In [58]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''

        try:
          tag = soup.find("h3", text='Minimum qualifications:').parent.find("ul")
          if tag:
            s = s + tag.text
        except:
          print(i, 'no mini qual')
        
        try:
          tag = soup.find('h3', text='Preferred qualifications:').parent.find("ul").findNextSibling('ul')
          if tag:
            s = s + tag.text
        except:
          print(i, 'no pref qual')

        try:
          tag = soup.find("div", {'id': 'accordion-responsibilities'})
          if tag:
            d = tag.text
        except:
          print(i, 'no description')
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [59]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

10 no mini qual
10 no pref qual
30 no mini qual
30 no pref qual
255 no mini qual
255 no pref qual
365 no mini qual
365 no pref qual
594 no mini qual
594 no pref qual
1029 no mini qual
1029 no pref qual
1030 no mini qual
1030 no pref qual
1031 no mini qual
1031 no pref qual
1033 no mini qual
1033 no pref qual
1034 no mini qual
1034 no pref qual
1035 no mini qual
1035 no pref qual


In [63]:
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

In [64]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'Google'
df = df.iloc[:, [4, 0, 1, 2, 3]]
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Google,"Software Engineer, Infrastructure, Looker Pro...",Experience in Site Reliability Engineering (SR...,https://careers.google.com/jobs/results/920102...,Design or develop features related to orchestr...
1,Google,Regional Legal Counsel,"Bachelor's degree of Law, comparable law degre...",https://careers.google.com/jobs/results/825474...,Work as part of the India Legal team advising ...
2,Google,"Scaled Customer Engineer, Google Cloud (Polis...",Bachelor's degree in Computer Science or equiv...,https://careers.google.com/jobs/results/107563...,Work closely with small and medium-sized busin...
3,Google,"Technical Program Manager Intern, Summer 2023","Currently pursuing a Bachelor’s, Master’s, or ...",https://careers.google.com/jobs/results/755884...,Perform specific responsibilities which vary b...
4,Google,"Senior Android Software Engineer, Fitbit Devi...","Bachelor's degree in Computer Science, a relat...",https://careers.google.com/jobs/results/136197...,"Design, develop, document, test, deploy, maint..."


In [65]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')
print(len(empty_mini_q))

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)
df.shape

(1025, 5)

In [68]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
df_nodup.shape

(1025, 5)

In [69]:
df_nodup.to_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\google_all_jobs_cleaned.csv')